# Gender and acceptance dataset cleaning pipeline (2nd iteration)

### Key changes:

Added features:
* Redefining all functions to include 2 new variables metric and status. Both variables determine which kind of dataset that will be futher analysed
    * Metric can be defined as gender, age or acceptance. 
    * Status can be defined as applicants and admitted
    * year define the data selection year
* Adding 'acceptance' and 'applicants' datasets to the cleaning pipeline


Modifications:
* Modifying the dataframe creater script to automatically drop all rows before "instnr" which is the first column of the wanted header.
* Replacing all null values with 0
* Changing all columns and table names from Danish to English 


## 1) Cleaning datasets from 2016 to 2022

Importing Libraries

In [1]:
# Importing libraries
import mysql.connector
from mysql.connector import Error
import os
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 50)
import sys
import math


1) Creating a function that define a dataframe based on the .csv file name {status}_{metric}_{yaer}. Then detecting the location of 'instnr' and dropping all rows before that and then setting 'instnr' row as the new header. 
2) Cleaning data by removing studie start from all academic majors. That allow us to perform calculations on their values later. 
3) Exporting and creating new folder based on the filename, and overwriting matching files.
4) Executing the function clean_and_save_dataframe with the preferred metrics

In [ ]:
# Creating a function to clean and save the dataframe
def clean_and_save_dataframe(year, metric, status, df_2022=None):
    csv_file = f"{metric}/{status}_{metric}_{year}.csv"
    
    # Read the CSV file without headers
    df = pd.read_csv(csv_file, header=None)

    # Find the index of the row containing 'InstNr'
    instnr_index = df[df.applymap(lambda x: x == '1000').any(axis=1)].index[0]

    # Drop all the rows before 'InstNr'
    df.drop(df.index[:instnr_index], inplace=True)

    # Define the new header
    if metric == 'gender' and year in range(2009, 2023):
        headers = ['instnr', 'instname', 'optnr', 'optname', 'male', 'female', 'total']
    
    
    elif metric == 'acceptance' and year in range(2009, 2014):
        headers = ['instnr', 'instname', 'optnr', 'optname', 'total', 'unknown', 'stx', 'hf', 'hhx', 'htx', 'ib', 'gif', 'afi', 'eud', 'andet', 'ib_udland', 'udland', 'eux']


    
    elif metric == 'acceptance' and year in range(2014, 2018) and year != 2016:
        headers = ['instnr', 'instname', 'optnr', 'optname', 'total', 'unknown', 'stx', 'hf', 'hhx', 'htx', 'ib', 'gif', 'afi', 'eud', 'andet', 'ib_udland', 'udland', 'eux']

    
    elif metric == 'acceptance' and year in range(2016, 2023) and year != 2017:
        headers = ['instnr', 'instname', 'optnr', 'optname', 'total', 'unknown', 'stx', 'hf', 'hhx', 'htx', 'ib', 'gif', 'afi', 'eud', 'andet', 'ib_udland', 'udland', 'eux', 'eux_1', 'faroese', 'greenlandic']
    
    else:
        print(f'Intervals {year}, {metric} or {status} not correct')
        sys.exit()

    # Add the new header to the dataframe
    df.columns = headers

    # Reset the index
    df.reset_index(drop=True, inplace=True)

    # create a boolean mask that identifies the rows containing 'I alt'
    mask = df.apply(lambda x: x.str.contains('I alt')).any(axis=1)

    # select only the rows that do not match the mask and drop all other rows
    df = df[~mask]

    # Drop rows with NaN in 'instnr' column
    df = df.dropna(subset=['instnr', 'instname', 'optnr', 'optname'])

    # Drop all rows containing E-læring
    df = df[~df['optname'].str.contains('E-læring')]

    # Drop rows with NaN in 'instnr' column
    df = df.dropna(subset=['instnr', 'instname', 'optnr', 'optname'])

    # Replace all null values with 0
    df = df.fillna(0)

    # Defining a function that converts to the dtyoe of the column to the corresponding header 
    def dtype_converter(df):
        for col in df.columns:
            if col in ['instnr', 'optnr', 'male','year', 'female', 'total', 'unknown', 'stx', 'hf', 'hhx', 'htx', 'ib', 'gif', 'afi', 'eud', 'andet', 'ib_udland', 'udland', 'eux', 'eux_1', 'faroese', 'greenlandic']:

              # convert float columns to int
                df[col] = df[col].apply(lambda x: math.floor(float(x))).astype(int)

                # Converting columns dtype to string
            elif col in ['instname', 'optname']:
                df[col] = df[col].astype(str)

    # Converts the columns to the corresponding dtype            
    dtype_converter(df)

    # Cleaning data by removing studie start
    df['optname'] = df['optname'].str.replace(', Studiestart: sommerstart', '')
    df['optname'] = df['optname'].str.replace(', Studiestart: vinterstart', '')
    df['optname'] = df['optname'].str.replace(', Studiestart: sommer- og vinterstart', '')
    df['optname'] = df['optname'].str.replace(', Study start: Summer start', '')
    df['optname'] = df['optname'].str.replace(', Study start: Winter start', '')
    
    # Gender data cleaning 
    if metric == 'gender':
        # Grouping and aggregating and adding matching academic majors students together
        df = df.groupby(['instnr', 'instname', 'optname']).agg(
            optnr=('optnr', 'max'),
            total=('total', 'sum'),
            male=('male', 'sum'),
            female=('female', 'sum')
            ).reset_index()
        
        if year > 2015:
        # Dropping everything after the last ',' This will result in removing the location
            df['optname'] = df['optname'].str.rsplit(',', n=1, expand=True)[0]

        # Updating table by adding a new column with the corresponding year
        df.insert(loc=1, column='year', value=year)
        
        # Reindexing the columns 
        df = df.reindex(columns=['instnr','year', 'instname', 'optnr', 'optname', 'male', 'female', 'total'])

    
    # Defining a file exporter function
    def file_exporter(df):
        folder_path = f'./{metric}_cleaned/'
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        file_path = folder_path + f'{status}_{metric}_clean_{year}.csv'
        df.to_csv(file_path, index=False)

    # Creating a dataframe to sort all unwanted educations and renaming all Academic Majors to 2022 new names 
    if year == 2022:
        file_exporter(df)
        df_2022 = df[['optnr', 'optname','instname']].rename(columns={'optname': 'optname_2022', 'instname' : 'instname_2022'})
        return df_2022
        
   
    # merge the DataFrames on the 'optnr' column if not 2022 and deleting all NaN rows
    if year != 2022:
        merged_df = pd.merge(df_2022, df, on='optnr', how='left')

        # Dropping Null values 
        df = merged_df.dropna(subset=(['optname']))

        # Dropping Name Axis 'old'
        df = df.drop(['optname','instname'], axis=1)

        # Renaming to the 'new' optname column 
        df = df.rename(columns={'optname_2022': 'optname', 'instname_2022': 'instname'})

        # Converting to the right corresponding dtype
        dtype_converter(df)

        # Reindexing the columns to the desired order
        df = df.reindex(columns=['instnr','year', 'instname', 'optnr', 'optname', 'male', 'female', 'total'])

        # Export cleaned csv files
        file_exporter(df)
    
    # Displaying all functions
    # display(df)

# Global dataframe
df_2022 = clean_and_save_dataframe(2022, 'gender', 'applicants')

# Iterate through the years, metrics and statuses and process each name and variant
metrics = ['gender']

statuses = ['applicants',
            'admitted'
            ]

years = [2022] + list(range(2009, 2022)) 

# Loop through all years and call table_creator() for each combination of metric and status
for year in (years):
    for metric in metrics:
        for status in statuses:
            if year == 2022:
                # Assign the result of the function call to df_2022
                df_2022 = clean_and_save_dataframe(year, metric, status)
            else:
                # Call the function with the current df_2022
                clean_and_save_dataframe(year, metric, status, df_2022)

# Creating acceptance table
clean_and_save_dataframe(2022, 'acceptance', 'admitted')


## 2) Connecting to SQL server

Defining server connection

In [2]:
# Creating a reuseable function that connect to mysql server, using host_name, user_name, user_password
def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection 
create_server_connection('localhost', 'root', '1234')

MySQL Database connection successful


### 2.1) Creating database and connecting

Creating database

In [3]:
# Defining a function that create a new data base
def database(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        if query == create:
            print(f"Database {database_name} created successfully")
        elif query == drop:
            print(f"Database {database_name} dropped successfully")
    except Error as err:
        print(f"Error: '{err}'")

# 1. Connecting to the SQL server and defining the connection
connection = create_server_connection('localhost', 'root', '1234')

# Define database name
database_name = 'university_distibution2'

# 2. Drop existing database
drop = f"Drop DATABASE if exists {database_name}"

# 3. Creating the SQL query that will create a database
create = f"CREATE DATABASE {database_name}"

# 4. Using the function to create the database¨
database(connection, drop)
database(connection, create)

MySQL Database connection successful
Database university_distibution2 dropped successfully
Database university_distibution2 created successfully


Connecting to database

In [3]:
# Defining a function that connects to the database 
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

# Define database name
database_name = 'university_distibution2'

# Connecting to the database
connection = create_db_connection('localhost', 'root','1234', database_name)


MySQL Database connection successful


Defining execute functions

In [4]:
# Defining a query execution function
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

Creating tables 

In [6]:
# Creating table gender
def table_creator(year, metric, status):
    if metric == 'gender':
        create_table = f"""
        CREATE TABLE {status}_{metric}_{year} (
            InstNr varchar(200),
            Year varchar(200),
            InstName varchar(200),
            OptNr varchar(200),
            OptName varchar(200),
            Male int,
            Female int,
            Total int   
        );
        """
       
    
    elif metric == 'acceptance' and year == 2022:
        create_table = f"""
        CREATE TABLE {status}_{metric}_{year} (
            InstNr int null,
            InstName varchar(200),
            OptNr int null,
            OptName varchar(200),
            Total int null,
            Uoplyst int null,
            STX int null,
            HF int null,
            HHX int null,
            HTX int null,
            IB int null,
            GIF int null,
            AFI int null,
            EUD int null,
            ANDET int null,
            IB_UDLAND int null,
            UDLAND int null,
            EUX int null,
            EUX_1 int null,
            Faroese int null,
            GREENLANDIC int null
        );
    
            """
    else:
        print('Error')
        
    # Execute our defined query   
    execute_query(connection, create_table)    


# Connect to the Database
connection = create_db_connection("localhost", "root", "1234", database_name)

# Iterate through the years from 2016 to 2022 and process each name and variant
metrics = ['gender']

statuses = ['admitted', 
            'applicants'
            ]
years = range(2009, 2023)

# Loop through all years and call table_creator() for each combination of metric and status
for year in years:
    for metric in metrics:
        for status in statuses:
            table_creator(year, metric, status)

# Creating acceptance table
table_creator(2022, 'acceptance', 'admitted')


MySQL Database connection successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful


## 3) Loading data from .csv into tables from 2016-2022

In [7]:
# Enabling file imports
set_global = """
SET GLOBAL local_infile=1;
"""
# Connect to the Database
connection = create_db_connection("localhost", "root", "1234", database_name)
execute_query(connection, set_global)

MySQL Database connection successful
Query successful


Loading data into tables based on conditions status, metric and year.

In [8]:

# Creating a function that loads .csv into tables with the corresponding year
def csv_loader(year, metric, status):
    load_csv_gender = f"""
    LOAD DATA INFILE 
        'C:/Projects/SQL_Portfolio project/Videre_gaaende_uddannelser/python/{metric}_cleaned/{status}_{metric}_clean_{year}.csv'
    INTO TABLE 
        {status}_{metric}_{year}
    FIELDS TERMINATED BY ','
    ENCLOSED BY '"'
    LINES TERMINATED BY '\r\n'
    IGNORE 1 LINES;
    """
    # Executing defined query
    execute_query(connection, load_csv_gender) 
    
# Connect to the Database
connection = create_db_connection("localhost", "root", "1234", database_name)

# Iterate through the years from 2016 to 2022 and process each name and variant
metrics = ['gender']

statuses = ['admitted', 
            'applicants'
            ]

years = range(2009, 2023)

# Loop through all years and call gender_dist_creator() for each combination of metric and status
for year in years:
    for metric in metrics:
        for status in statuses:
            csv_loader(year, metric, status)

csv_loader(2022, 'acceptance', 'admitted')

MySQL Database connection successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful


## 4) Data analysis

### 4.1) Academic Major Gender Distibution

Creating a new view that group all of the matching educations(optname) and taking the sum of the students, and showing gender distibution in numbers and procentage share

In [18]:
#Creating a function that execute a query select and calculate the gender distibution in pct.
def gender_dist_creator(year, status):
    create_gender_view = f"""
    CREATE VIEW {status}_gender_dist_{year} AS
    Select 
        instNr, 
        InstName,
        Max(optnr) as OptNr,
        OptName,
        sum(Total) as Total,
        sum(Male) as Male, 
        round(sum(Male)/sum(total)*100,2) as Male_pct,
        sum( Female) as Female, 
        round(sum(Female)/sum(total)*100,2) as Female_pct
    from {status}_gender_{year}
    group by 
        InstNr, 
        InstName, 
        OptName
    order by 
        InstNr;   
    """
    execute_query(connection, create_gender_view)

# Defining year and status 
years = range(2022, 2023)
statuses = [ #'applicants', 
            'admitted'
            ]

# Iterating throught years and statuses
for year in years:
    for status in statuses:
        gender_dist_creator(year, status)



Query successful


### 4.2) Merging all tables  

In [5]:
# Create a view that merges the admitted_gender tables
def table_merger(status):
    merge = f"""
        CREATE OR REPLACE VIEW {status}_gender_merged AS
        SELECT * FROM {status}_gender_2009
        UNION ALL
        SELECT * FROM {status}_gender_2010
        UNION ALL
        SELECT * FROM {status}_gender_2011
        UNION ALL
        SELECT * FROM {status}_gender_2012
        UNION ALL
        SELECT * FROM {status}_gender_2013
        UNION ALL
        SELECT * FROM {status}_gender_2014
        UNION ALL
        SELECT * FROM {status}_gender_2015
        UNION ALL
        SELECT * FROM {status}_gender_2016
        UNION ALL
        SELECT * FROM {status}_gender_2017
        UNION ALL
        SELECT * FROM {status}_gender_2018
        UNION ALL
        SELECT * FROM {status}_gender_2019
        UNION ALL
        SELECT * FROM {status}_gender_2020
        UNION ALL
        SELECT * FROM {status}_gender_2021
        UNION ALL
        SELECT * FROM {status}_gender_2022;
    """
    execute_query(connection, merge)

# Defining status
statuses = [ 'applicants', 
            #'admitted'
            ]

# Iterating throug different status
for status in statuses:
    table_merger(status)


Query successful


### 4.2) University Gender Distibution

Grouping all academic majors from the same university and calculating the gender distibution

In [ ]:
# Creating a function that groups universities and calculate the percentage distibution
def gender_dist_uni(year, status):
    gender_uni_view = f"""
    CREATE VIEW {status}_gender_dist_uni_{year} as
    select 
        instnr,
        instname,
        sum(total_{year}) as total_{year},
        sum(male_{year}) as male_{year},
        round(sum(male_{year})/sum(total_{year})*100,2) as male_pct_{year},
        sum(female_{year}) as female_{year},
        round(sum(female_{year})/sum(total_{year})*100,2) as female_pct_{year}
    from {status}_gender_dist_{year}
    group by
        instnr, instname
    order by 
        instnr;

    """
    execute_query(connection, gender_uni_view)

# Defining year and status 
years = range(2016, 2023)
statuses = ['applicants', 'admitted']

# Iterating throught years and statuses
for year in years:
    for status in statuses:
        gender_dist_uni(year, status)



Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful


### 4.3) Total Gender distibution

Grouping all universities and calculating the total distibution of students on the indiviual universities

In [12]:
# Defining a function that execute the following SQL Query
def student_dist(year, status):
    student_dist_view = f"""
    CREATE VIEW {status}_total_student_{year} as
    SELECT 
        instnr,
        instname,
        round(sum(total_{year}) / (select sum(total_{year}) from {status}_gender_{year}) * 100,2) as pct_{year},
        SUM(total_{year}) AS Total_{year}
    FROM {status}_gender_{year}
    GROUP BY 
        instnr,
        instname
    Order by total_{year} desc;

    """
    execute_query(connection, student_dist_view)
    
# Defining year and status 
years = range(2016, 2023)
statuses = ['applicants', 'admitted']

# Iterating throught years and statuses
for year in years:
    for status in statuses:
        student_dist(year, status)

Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
